In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import os,time
import glob,sys

from astropy.io import fits
from astropy.visualization import ZScaleInterval

from RetrieveSource import *
from EstimateBackground import *
from TractorTools import *
from Heasarc import *
from PSF import *

In [2]:
def vmin(data):
    return ZScaleInterval().get_limits(data)[0]
def vmax(data):
    return ZScaleInterval().get_limits(data)[1]

In [5]:
# Step 0: Get Files from Path

# Observations Ids for SMC_# Survey [50 total]
smc_observation_id = np.arange(40415,40465,1).astype(str)

# Observations Ids for LMC_Survey_# Survey [162 total]
lmc_observation_id = np.arange(45422,45587,1).astype(str)

# Pick which galaxy to do 
galaxies=['LMC','SMC']
gal = galaxies[0]                       # could automate this. 

# Pick an observation ID
observation_ids = lmc_observation_id[49]
print(observation_ids)

path = '../Results/Photometry'

def get_hdu_cat_from_file(path,observation_id):
    # TO DO: 
    # Need to update this function so it does all segments

    file = f'sw000*{observation_id}*.new'
    filenames = glob.glob(os.path.join(f'Data/UV/{gal}',file))
    print(f"%s files found for id {observation_id}" % len(filenames))

    # If it has all 3 filters --- Might not be the best idea.... maybe should just do everything.
    #if len(filenames) == 3: 
    hdus = [fits.open(file)[0] for file in filenames]
    names = [file.split('/')[-1].split('.')[0] for file in filenames]
    catfiles = [f"Data/UV/{gal}/{name}.full.dat" for name in names]

    return hdus,catfiles

    #print(f"Warning: Not format structure that you thought {observation_id}") 
    #return [1],[1]
        

def model_field(observation_id):
    
    print('Attempting ObsID: ', observation_id)
    t1 = time.time()
    
    # If this obsid has not been run before make it it's own folder
    if not os.path.exists(f"../Results/Photometry/{observation_id}"):
        os.mkdir(f"../Results/Photometry/{observation_id}/")   
        
    # To save time, retrieve sources has been run on most fields saving a catalog of sources per field 
    # Load the right sources within the observation_id field
    hdu,catname = get_hdu_cat_from_file(path,observation_id) 

    #if len(hdu) < 3: continue
    
    meta = {}; bkgd = {}; TractorObjects = {}; 
    Magnitudes = {}; by_filter = {}


    for i,h in enumerate(hdu):
        uvfilter = h.header['FILTER']
        shape = np.shape(h.data)
    
        # Step 1: Get Zaritsky Coordinates and Catalog 
        tz = time.time()
    
        print('---Getting Catalog---')

        catalog = catname[i]
        meta[uvfilter] = get_meta().with_hdu(h,
                                             optical_catalog=catalog,
                                             xdim=[0,shape[1]],
                                             ydim=[0,shape[0]],
                                             cutoff=20.5,
                                             fits_origin=0,
                                             astrometry=True)
    
        print("Time To Run Retrieve Source: ",(time.time()-tz)/60)
    

    
        # Step 2: Error Estimation
        bkgd[uvfilter] = BkgdEstimator(meta[uvfilter],n_pix = [20,20])
    

        # Step 3: Calculate PSF
        pix_scale = np.abs(meta[uvfilter].cdelt)*3600.
        psf_object = psf_fit(pixel_per_arsecond = 1/pix_scale,
                             uvfilter = uvfilter, 
                             width = 23).psf
    
    
        # Step 4: Run Tractor 
        t2 = time.time()
        print('---Starting to Run Tractor---')
        fwhm = 2.5
        # 2.355 is the conversion from fwhm to sigma
        sigma =  fwhm/2.355
        # Restrict by a factor of 4.
        sigma = sigma/4.
        
        TractorObjects[uvfilter] = PhotometryTools(meta[uvfilter],
                            psf_filename = psf_object,
                            fits_table_sname = f"{path}/{observation_id}/{observation_id}_{uvfilter}_{exp_str}_1_1.fits", # Should change 1_1 to include all segments!
                            background = bkgd[uvfilter],
                            fit_positions = sigma,
                            threshold = 1.5)
        print("Time To Run Tractor: ",(time.time()-t2)/60)
    
        # Step 5: Run Heasarc 
        t3 = time.time()
        Magnitudes[uvfilter] = HeasarcRoutines(f"{path}/{observation_id}/{observation_id}_{uvfilter}_{exp_str}_1_1.fits",uvfilter)# Should change 1_1 to include all segments!
        print("Time to Run Heasarc: ",(time.time()-t3)/60)
    
        # Step 6: Format CSV
        t4 = time.time()    
    
        # Get csv created in Heasarc 
        df = pd.read_csv(f'{path}/{observation_id}/{observation_id}_{uvfilter}_{exp_str}_1_1.csv')# Should change 1_1 to include all segments!

        labels = [uvfilter + '_'+ key for key in df.keys()]
        d = {}
        for old,new in zip(df.keys(),labels):
            d[old] = new
        df = df.rename(columns=d)

        print(f'Joining Optical catalog with {uvfilter}')
        by_filter[uvfilter] = pd.merge(meta[uvfilter].catalog,df,left_on='KEY',right_on=f'{uvfilter}_KEY',how="inner")
        print("Time to Create CSV by Filter: ",(time.time()-t4)/60)
        
    
        by_filter[uvfilter] = by_filter[uvfilter][by_filter[uvfilter][uvfilter+'_MAG_ERR'] < 0.35]

        return TractorObjects

45471


In [7]:
model_field(40430)

Attempting ObsID:  40430
0 files found for id 40430
